# CIGRE MV with PV Inverters

## Powerflow

### Run simulation

In [ ]:
import requests
import glob

def download_grid_data(name, url):
    with open(name, 'wb') as out_file:
        content = requests.get(url, stream=True).content
        out_file.write(content)

url = 'https://git.rwth-aachen.de/acs/public/grid-data/cim-grid-data/-/raw/master/CIGRE_MV/NEPLAN/CIGRE_MV_no_tapchanger_noLoad1_LeftFeeder_With_LoadFlow_Results/Rootnet_FULL_NE_28J17h'
filename = 'CIGRE-MV'
download_grid_data(filename+'_EQ.xml', url+'_EQ.xml')
download_grid_data(filename+'_TP.xml', url+'_TP.xml')
download_grid_data(filename+'_SV.xml', url+'_SV.xml')

files = glob.glob(filename+'_*.xml')
print(files)

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import matplotlib.pyplot as plt
import dpsimpy

In [ ]:
sim_name = 'PF_CIGRE_MV_withDG'

### Powerflow for Initialization

In [ ]:
dpsimpy.Logger.set_log_dir('logs/' + sim_name)
reader = dpsimpy.CIMReader(sim_name, dpsimpy.LogLevel.debug, dpsimpy.LogLevel.debug)
system = reader.loadCIM(50, files, dpsimpy.Domain.SP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.PVNode)
pv_active_power = 50e3 * int(int(4319.1e3 / 50e3) / 9)
pv_reactive_power = np.sqrt(np.power(pv_active_power / 1, 2) - np.power(pv_active_power, 2))

for n in range(3, 12):
    connection_node = system.node('N' + str(n))
    pv = dpsimpy.sp.ph1.AvVoltageSourceInverterDQ('pv_' + connection_node.name(), 'pv_' + connection_node.name(), dpsimpy.LogLevel.debug, True)
    pv.set_parameters(sys_omega=2 * np.pi * 50, sys_volt_nom=1500, p_ref=pv_active_power, q_ref=pv_reactive_power)
    pv.set_controller_parameters(Kp_pll=0.25 / 10, Ki_pll=2 / 1000,
                                Kp_power_ctrl=0.001 / 10, Ki_power_ctrl=0.08 / 1000 ,
                                Kp_curr_ctrl=0.3 / 10, Ki_curr_ctrl=10 / 1000, omega_cutoff=2*np.pi*50)
    pv.set_filter_parameters(Lf=0.002,Cf=789.3e-6,Rf=0.1,Rc=0.1)
    pv.set_transformer_parameters(nom_voltage_end_1=20e3, nom_voltage_end_2=1500, rated_power = 5e6,
                                ratio_abs=20e3 / 1500, ratio_phase=0,resistance=0, inductance=0.928e-3)
    pv.set_initial_state_values(p_init=450000.716605, q_init=-0.577218, phi_d_init=3854.197405 * 1000, phi_q_init=-0.003737 * 1000, gamma_d_init=128.892668 * 1000, gamma_q_init=23.068682 * 1000)
    system.add(pv)
    system.connect_component(pv, [connection_node])


logger = dpsimpy.Logger(sim_name)
for node in system.nodes:
    logger.log_attribute(node.name() + '.V', 'v', node)

sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.debug)
sim.set_system(system)
sim.set_time_step(1.0)
sim.set_final_time(2.0)
sim.set_domain(dpsimpy.Domain.SP)
sim.set_solver(dpsimpy.Solver.NRP)
sim.do_init_from_nodes_and_terminals(True)

sim.add_logger(logger)
sim.run()


### Read results

In [ ]:
modelName = 'PF_CIGRE_MV_withDG'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim = read_timeseries_csv(dpsim_result_file)

### Plot results

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name != 'N0.V':
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()